# causality
Python package

In [1]:
import numpy
import pandas as pd

from causality.inference.search import IC
from causality.inference.independence_tests import RobustRegressionTest

# generate some toy data:
SIZE = 2000
x1 = numpy.random.normal(size=SIZE)
x2 = x1 + numpy.random.normal(size=SIZE)
x3 = x1 + numpy.random.normal(size=SIZE)
x4 = x2 + x3 + numpy.random.normal(size=SIZE)
x5 = x4 + numpy.random.normal(size=SIZE)

# load the data into a dataframe:
X = pd.DataFrame({'x1' : x1, 'x2' : x2, 'x3' : x3, 'x4' : x4, 'x5' : x5})

# define the variable types: 'c' is 'continuous'.  The variables defined here
# are the ones the search is performed over  -- NOT all the variables defined
# in the data frame.
variable_types = {'x1' : 'c', 'x2' : 'c', 'x3' : 'c', 'x4' : 'c', 'x5' : 'c'}

# run the search
ic_algorithm = IC(RobustRegressionTest)
graph = ic_algorithm.search(X, variable_types)

/Users/amir.ziai/anaconda/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [12]:
X.head()

,x1,x2,x3,x4,x5
0,-1.406695,-0.517982,0.142170,-1.816308,-2.210177
1,0.350395,0.899873,-0.267734,0.078581,1.170726
2,-0.861684,-0.558598,-0.075510,1.639826,2.057733
3,-0.334222,0.539190,0.020224,0.116004,0.310686
4,1.154677,0.647006,1.194869,2.751039,3.158303


In [2]:
graph.edges(data=True)

[('x3', 'x4', {'arrows': ['x4'], 'marked': False}),
 ('x3', 'x1', {'arrows': [], 'marked': False}),
 ('x1', 'x2', {'arrows': [], 'marked': False}),
 ('x5', 'x4', {'arrows': ['x5'], 'marked': True}),
 ('x4', 'x2', {'arrows': ['x4'], 'marked': False})]

In [3]:
from causality.estimation.adjustments import AdjustForDirectCauses
from networkx import DiGraph

g = DiGraph()

g.add_nodes_from(['x1','x2','x3','x4', 'x5'])
g.add_edges_from([('x1','x2'),('x1','x3'),('x2','x4'),('x3','x4')])
adjustment = AdjustForDirectCauses()

In [5]:
print (adjustment.admissable_set(g, ['x2'], ['x3']))

{'x1'}


In [6]:
from causality.estimation.nonparametric import CausalEffect

In [7]:
admissable_set = adjustment.admissable_set(g,['x2'], ['x3'])

In [8]:
effect = CausalEffect(X, ['x2'], ['x3'], variable_types=variable_types, admissable_set=list(admissable_set))

In [9]:
x = pd.DataFrame({'x2' : [0.], 'x3' : [0.]})

In [10]:
effect.pdf(x)

0.269858323490185